# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


In [ ]:
# We import all the necessary libraries and tools
!pip install selenium
!pip install webdriver-manager
import time
import shutil
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_info(index):
    # Configura las opciones de Chrome
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.binary_location = 'C:/Program Files/Google/Chrome/Application/chrome.exe'

    # Obtiene la ruta del controlador usando ChromeDriverManager
    driver_path = ChromeDriverManager().install()

    # Crea una instancia de webdriver con ChromeOptions
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()

    url = 'https://infogob.jne.gob.pe/Eleccion'
    driver.get(url)

    elecciones_button_selector = '#section > div.content-busqueda > div.cajas-busqueda.fondo-grisclaro > div.form-group.row > div:nth-child(1) > div > div.text'
    elecciones_button = driver.find_element(By.CSS_SELECTOR, elecciones_button_selector)
    elecciones_button.click()

    time.sleep(3)